In [1]:
import pandas as pd
import numpy as np
import requests
import folium

#!pip install Beautifulsoup4
#!pip install lxml
from bs4 import BeautifulSoup

## Using beautiful soup to get the html page
I Used Inspect Element on the wikipedia webpage and found out that the whole table was contained under the tbody column. So i used the find function to find the first tbody tag and extract all the information inside it.

In [2]:
source = requests.get(
    'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

summary = soup.find("tbody")


### Creating a Dataframe

In [3]:
column_name = ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(columns=column_name)
df

,PostalCode,Borough,Neighbourhood


### Finding the table
When i used the developers console on the webpage i found that the table rows used the tag td so i extracted all the text starting with the td tag. 

In [4]:
rows = summary.find_all('td')

In [5]:
n=len(rows)
n

540

### Extracting the data and putting it in the dataframe.

In [6]:
for a in range(0, 539, 3):
    postalcode = rows[a].text.rstrip()
    borough = rows[a+1].text.rstrip()
    neigh = rows[a+2].text.rstrip()
    df = df.append({'PostalCode': postalcode,
                    'Borough': borough,
                    'Neighbourhood': neigh}, ignore_index=True)

In [7]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Removing the unassigned 

In [8]:
for a in df.index:
    if(df['Borough'][a] == 'Not assigned'):
        df.drop(index=a, axis=0, inplace=True)

In [16]:
x = df
newdf = pd.DataFrame(columns=column_name)
x.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Removing recurring postal codes and adding the neighborhoods 

In [20]:
y = x['PostalCode'].unique()
for value in y:
    flag = 0
    for a in df.index:
        if flag == 0:
            if(value == df['PostalCode'][a]):
                postal_code = df['PostalCode'][a]
                Bor = df['Borough'][a]
                neighbourhood = df['Neighbourhood'][a]
                newdf = newdf.append({'PostalCode': postal_code,
                                      'Borough': Bor,
                                      'Neighbourhood': neighbourhood}, ignore_index=True)
                prime = 0
                flag = 1
                continue
        if flag == 1:
            if(value == df['PostalCode'][a]):
                orig = df['Neighbourhood'][a]
                newdf['Neighbourhood'][prime] = newdf['Neighbourhood'][prime]+","+orig
    prime = prime+1

In [21]:
newdf.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
newdf.shape

(103, 3)